<h2> Tal Davidi : 208871376 </h2>
<h4> Skoda 2003-2024 Data</h4>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


### First Part - Get the Skoda Data from web

In [ ]:
def get_car(keywords):
    car_list = list()
    page = 1
    
    while True:
        url = f"https://www.ad.co.il/car?keyword={keywords}&pageindex={page}"
        
        try:
            response = requests.get(url)
            if response.status_code != 200:
                return None
            
            results_page = BeautifulSoup(response.content, 'html.parser')
            cars = results_page.find_all('div', {'class': "card-body p-md-3"})
            if keywords.split()[0] not in cars[0].get_text():
                break
            
            for car in cars:
                car_link = "https://www.ad.co.il" + car.find('a').get('href')
                car_name = car.find('a').get_text()
                
                try:
                    car_price = car.find('div', {'class': 'price ms-1'}).get_text()
                except:
                    car_price = ''
                    
                car_list.append((car_name, car_link, car_price))
            
            page += 1
           # time.sleep(1)  # השהייה של 1 שניות בין הבקשות
            
        except Exception as e:
            print(f"An error occurred on page {page}: {e}")
            break
            
    
    return car_list


In [ ]:
get_car("סקודה")

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_car_details(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return None
        
        car_details = {}
        car_page = BeautifulSoup(response.content, 'html.parser')
        table_rows = car_page.find_all('tr')
        
        for row in table_rows:
            cols = row.find_all('td')
            if len(cols) == 2:
                key = cols[0].get_text().strip()
                value = cols[1].get_text().strip()
                car_details[key] = value
        
        photos = car_page.find_all('div', {'class': "justify-content-center px-1"})
        num_of_photos = len(photos)
        car_details["Pic_num"] = num_of_photos
        
        dates = car_page.find_all('div', {'class': "px-3"})
        if len(dates) >= 2:
            car_details["Cre_date"] = dates[0].get_text().strip()
            car_details["Repub_date"] = dates[1].get_text().strip()
        
        description = car_page.find('p', {'class': "text-word-break"})
        if description :     
            car_details["Description"] = description.get_text().strip()
        else:
            car_details["Description"] = ''
            
        
        
        
        return car_details
    except Exception as e:
        print(f"An error occurred while fetching details from {url}: {e}")
        return None

In [ ]:
get_car_details("https://www.ad.co.il/ad/16186942")

In [ ]:
def get_all_cars(keywords):
    results = list()
    all_cars = get_car(keywords)
    for car in all_cars:
        car_dict = {}
        car_dict['name'] = car[0]
        car_dict['price'] = car[2]
        details = get_car_details(car[1]) 
        for key,value in details.items() :
            car_dict[key] = value
     
        results.append(car_dict)
    return(results)

In [ ]:
data = get_all_cars("סקודה")

### Second Part - Order the data in pandas

In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
df = df[df['name'].str.contains('סקודה')]
df

In [ ]:
df.info()

#### Rename

In [ ]:
df = df[['name','price','ת. הילוכים','יד','נפח','סוג מנוע','ק"מ','צבע','בעלות קודמת','בעלות נוכחית','אזור','עיר','שנה','Pic_num','Cre_date','Repub_date','Description','טסט עד']]

In [ ]:
old_names = ['name','price','ת. הילוכים','יד','נפח','סוג מנוע','ק"מ','צבע','בעלות קודמת','בעלות נוכחית','אזור','עיר','שנה','Pic_num','Cre_date','Repub_date','Description','טסט עד']
new_names =  ['Name','Price','Gear','Hand','Engine_capacity','Engine_type','Km','Color','Prev_ownership','Curr_ownership','Area','City','Year','Pic_num','Cre_date','Repub_date','Description','Test']
name_mapping = dict(zip(old_names, new_names))
df = df.rename(columns=name_mapping)
df

### Dtypes Converting :

In [ ]:
df = df.astype("string")
df.dtypes

In [ ]:
df[['Manufacturer', 'Model']] = df['Name'].str.split(' ', 1, expand=True)

In [ ]:
df = df[[ 'Manufacturer','Model','Price','Gear','Hand','Capacity_engine','Engine_type','Km','Color','Prev_ownership','Curr_ownership','Area','City','Year','Pic_num','Cre_date','Repub_date','Description','Test']]

In [ ]:
df

In [ ]:
df['Price'] = df['Price'].str.replace(',', '')
df['Price'] = df['Price'].str.replace('₪', '')
df['Capacity_engine'] = df['Capacity_engine'].str.replace(',', '')
df['Km'] = df['Km'].str.replace(',', '')
df['Cre_date'] = df['Cre_date'].str.replace('תאריך יצירה:', '')
df['Repub_date'] = df['Repub_date'].str.replace('תאריך הקפצה אחרון:', '')

In [ ]:
df

#### int

In [ ]:
df['Year'] = df['Year'].astype(int)
df['Hand'] = df['Hand'].astype(int)

df['Km'] = pd.to_numeric(df['Km'], errors='coerce')
df['Km'] = df['Km'].astype('Int64')

df['Capacity_engine'] = df['Capacity_engine'].astype(int)
df['Pic_num'] = df['Pic_num'].astype(int)

some of columns in int32 and some of in int64.

In [ ]:
print(df.dtypes)

#### Dates :

In [ ]:
df['Cre_date'] = pd.to_datetime(df['Cre_date'])
df['Repub_date'] = pd.to_datetime(df['Repub_date'])

#### Category :

In [ ]:
lst = ['בנזין', 'דיזל', 'גז', 'היבריד', 'חשמלי']
df.loc[df['Engine_type'] == 'טורבו דיזל', 'Engine_type'] = 'דיזל'
df.loc[~df['Engine_type'].isin(lst), 'Engine_type'] = None
df['Engine_type'] = df['Engine_type'].astype('category')

Gear

In [ ]:
unique_values, counts = np.unique(df['Gear'], return_counts=True)
value_counts = dict(zip(unique_values, counts))
lst1 = list(value_counts.keys())

# Set values not in lst1 to None
df['Gear'] = df['Gear'].where(df['Gear'].isin(lst1), None)

# Convert the 'Gear' column to category type
df['Gear'] = df['Gear'].astype('category')

Prev_ownership

In [ ]:
lst2 = list(df['Prev_ownership'].unique())
df['Prev_ownership'] = df['Prev_ownership'].where(df['Prev_ownership'].isin(lst2), None)
df['Prev_ownership'] = df['Prev_ownership'].astype('category')

Curr_ownership

In [ ]:
lst2 = list(df['Curr_ownership'].unique())
df['Curr_ownership'] = df['Curr_ownership'].where(df['Curr_ownership'].isin(lst2), None)
df['Curr_ownership'] = df['Curr_ownership'].astype('category')

Test

In [ ]:
# Convert 'Test' column to datetime
df['Test'] = pd.to_datetime(df['Test'], errors='coerce')

# Get today's date
today = pd.Timestamp('today').normalize()

# Calculate the last day of the month for each date in the 'Test' column
df['End_of_month'] = df['Test'] + pd.offsets.MonthEnd(0)

# Calculate the number of days from today to the end of the month
df['Days_to_end_of_month'] = (df['End_of_month'] - today).dt.days

# Replace NaN values with None
df['Days_to_end_of_month'].where(df['Days_to_end_of_month'].notna(), None, inplace=True)

# Convert the 'Days_to_end_of_month' column to integer type
df['Test'] = df['Days_to_end_of_month'].astype('Int64')

# Drop the 'End_of_month' and 'Days_to_end_of_month' columns as they are no longer needed
df = df.drop(columns=['End_of_month', 'Days_to_end_of_month'])


In [ ]:
#check
df[["Test"]]

In [305]:
df = df[(df['Year'] >= 2017) & (df['Year'] <= 2024)]

In [306]:
df

,Manufacturer,Model,Price,Gear,Hand,Capacity_engine,Engine_type,Km,Color,Prev_ownership,Curr_ownership,Area,City,Year,Pic_num,Cre_date,Repub_date,Description,Test
0,סקודה,סופרב,122000,אוטומטית,2,1500,בנזין,70000,לבן פנינה,פרטית,פרטית,טבריה והסביבה,גבעת אבני,2021,8,2024-08-06,2024-12-06,"סקודה סופרב שנת 3/2021, דגם אמבישן, 150 כ""ס,מ...",291
1,סקודה,אוקטביה,50000,אוטומטית,2,2000,דיזל,155000,לבן,מונית,פרטית,אשדוד - אשקלון,אשדוד,2018,2,2024-11-06,2024-11-06,"סקודה אוקטביה 2018 מונית שעבר 155,000 ק״מ הרכב...",79
2,סקודה,אוקטביה ספייס,65000,אוטומטית,2,1500,בנזין,170000,לבן,חברה,פרטית,"ראשל""צ והסביבה",באר יעקב,2019,4,2024-05-24,2024-05-24,המנוע הכי מוצלח 1.5 עם 150 כ''ס סטיישן רכב מעו...,79
3,סקודה,אוקטביה,116000,אוטומטית,2,1800,בנזין,57000,שחור,פרטית,פרטית,ראש העין והסביבה,ראש העין,2019,6,2024-01-05,2024-01-05,סקודה קארוק סטייל שחורה נדירה. קילומטר נמוך מא...,201
10,סקודה,אוקטביה,65000,אוטומטית,1,1400,בנזין,<NA>,לבן,פרטית,פרטית,חולון - בת ים,בת ים,2018,5,2023-10-09,2024-11-03,שמורה היטה! במצב מעולה ללא תאונות רכב מפנק במיוחד,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,סקודה,סופרב,145000,אוטומטית,2,1800,בנזין,80000,חום,פרטית,פרטית,חיפה וחוף הכרמל,חיפה,2017,1,2021-12-14,2021-12-14,רכב שמור 80אלף ק״ם!! עם סטייג 1 תוכנת ניהול ל...,<NA>
255,סקודה,סופרב,220000,אוטומטית,2,2000,בנזין,70000,<NA>,<NA>,<NA>,תל אביב,תל אביב יפו,2019,0,2022-04-21,2022-04-21,רכב שמור עם ספר טיפולים. נאלץ למכור עקב קבלת ר...,<NA>
256,סקודה,ראפיד,59000,אוטומטית,2,1200,בנזין,150000,<NA>,<NA>,<NA>,חיפה וחוף הכרמל,חיפה,2017,0,2022-04-18,2022-04-18,"סקודה ראפיד שנת 2017, שמורה היטב ומטופלת קבוע,...",<NA>
257,סקודה,אוקטביה,45000,אוטומטית,1,1600,בנזין,40000,שחור,פרטית,פרטית,תל אביב,תל אביב יפו,2018,0,2022-02-28,2022-01-03,"דרושים לעבודות תמלול, קלדנות כתיבה, הזנת תוכן,...",<NA>


# הערות :

#### בחרתי את סדר העמודות בצורה זו שהיא הכי נוחה לעבודה עבורי

#### in the data i have col "Name" contain "סקודה אוקטביה" i can sep it by code bellow , but i prefer it like this.
#### the code : 
#### df[['Manufacturer', 'Model']] = df['Name'].str.split(n=1, expand=True)

In [307]:
# Save DataFrame to CSV
df.to_csv('Tal_DF.csv', index=False)